<a href="https://colab.research.google.com/github/khemsu/Data_Warehosuing_and_mining_labs/blob/main/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q1)

In [2]:
import pandas as pd
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

try:
    with open("/content/drive/MyDrive/DataMining/space (1).txt", "r") as f:
        lines = f.readlines()[1:]
except FileNotFoundError:
    print("Error: 'sports.txt' not found. Please check the file path.")
    raise

transactions_list = []
for line in lines:
    parts = line.strip().split(",")
    items = [item.strip() for item in parts[1:] if item.strip()]
    if items:
        transactions_list.append(items)


print("Sample transactions:", transactions_list[:5])

def get_support(item, transactions):
    count = sum(1 for transaction in transactions if set(item).issubset(set(transaction)))
    return count / len(transactions) if transactions else 0

def apriori(transactions, min_support):
    itemsets = []
    single_items = set(item for transaction in transactions for item in transaction)

    L1 = []
    for item in single_items:
        support = get_support([item], transactions)
        if support >= min_support:
            L1.append(([item], support))
    itemsets.extend(L1)
    if not L1:
        return itemsets

    k = 2
    prev_frequent = [[item for item in items] for items, _ in L1]

    while prev_frequent:
        candidates = []
        for i in range(len(prev_frequent)):
            for j in range(i + 1, len(prev_frequent)):
                itemset1, itemset2 = prev_frequent[i], prev_frequent[j]

                if itemset1[:k-2] == itemset2[:k-2]:
                    candidate = sorted(list(set(itemset1 + itemset2)))
                    if len(candidate) == k:

                        subsets = [sorted(list(c)) for c in combinations(candidate, k-1)]
                        if all(sorted(s) in prev_frequent for s in subsets):
                            candidates.append(candidate)

        candidates = [list(c) for c in set(tuple(c) for c in candidates)]
        if not candidates:
            break

        Lk = []
        for candidate in candidates:
            support = get_support(candidate, transactions)
            if support >= min_support:
                Lk.append((candidate, support))

        if not Lk:
            break
        itemsets.extend(Lk)
        prev_frequent = [[item for item in items] for items, _ in Lk]
        k += 1

    return itemsets




def calc_confidence_lift(frequent_itemsets, transactions):
    print("\n📈 Association Rules:")
    rules_found = False
    for itemset, support in frequent_itemsets:
        if len(itemset) >= 2:
            for i in range(1, len(itemset)):
                for antecedent in combinations(itemset, i):
                    antecedent = list(antecedent)
                    consequent = sorted(list(set(itemset) - set(antecedent)))
                    antecedent_support = get_support(antecedent, transactions)
                    consequent_support = get_support(consequent, transactions)
                    if antecedent_support > 0 and consequent_support > 0:
                        confidence = support / antecedent_support
                        lift = confidence / consequent_support
                        print(f"{antecedent} => {consequent} | support={support:.2f}, confidence={confidence:.2f}, lift={lift:.2f}")
                        rules_found = True
    if not rules_found:
        print("No association rules found. Try lowering min_support or check dataset.")

min_support = 0.1
frequent_itemsets = apriori(transactions_list, min_support)

print("\n📌 Frequent Itemsets (Support >= 0.2):")
for itemset, support in frequent_itemsets:
    print(f"{itemset} - support: {support:.2f}")

calc_confidence_lift(frequent_itemsets, transactions_list)

Sample transactions: [['Robotic Arm', 'Food Packets'], ['Sleeping Bag', 'Treadmill', 'Food Packets'], ['Robotic Arm', 'Space Suit', '3D Printer'], ['Food Packets', 'Carbon Dioxide Scrubbers'], ['Sleeping Bag', 'Space Suit']]

📌 Frequent Itemsets (Support >= 0.2):
['Carbon Dioxide Scrubbers'] - support: 0.24
['Space Suit'] - support: 0.32
['Treadmill'] - support: 0.28
['3D Printer'] - support: 0.28
['Food Packets'] - support: 0.40
['Sleeping Bag'] - support: 0.32
['Robotic Arm'] - support: 0.34
['Food Packets', 'Robotic Arm'] - support: 0.14
['Robotic Arm', 'Space Suit'] - support: 0.10
['Food Packets', 'Sleeping Bag'] - support: 0.10
['Food Packets', 'Space Suit'] - support: 0.10
['3D Printer', 'Sleeping Bag'] - support: 0.10
['Food Packets', 'Treadmill'] - support: 0.12
['Robotic Arm', 'Sleeping Bag'] - support: 0.10

📈 Association Rules:
['Food Packets'] => ['Robotic Arm'] | support=0.14, confidence=0.35, lift=1.03
['Robotic Arm'] => ['Food Packets'] | support=0.14, confidence=0.41, 

b)


In [ ]:
import pandas as pd
from itertools import combinations
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# ✅ 1. Load sports.txt
file_path = "/content/drive/MyDrive/DataMining/sports (1).txt"
transactions = []

try:
    with open(file_path, "r") as f:
        lines = f.readlines()[1:]  # Skip header
except FileNotFoundError:
    print("Error: File not found.")
    raise

for line in lines:
    parts = line.strip().split(",")
    items = [item.strip() for item in parts[1:] if item.strip()]
    if items:
        transactions.append(items)

print("Sample transactions:", transactions[:5])
total_tx = len(transactions)

# ✅ 2. Support function
def get_support(itemset, transactions):
    count = sum(1 for tx in transactions if itemset.issubset(set(tx)))
    return count / total_tx

# ✅ 3. Apriori algorithm
def apriori(transactions, min_support):
    item_counts = defaultdict(int)

    for tx in transactions:
        for item in tx:
            item_counts[frozenset([item])] += 1

    frequent_itemsets = {item: count for item, count in item_counts.items() if count / total_tx >= min_support}
    all_frequent = frequent_itemsets.copy()
    current_freq = list(frequent_itemsets.keys())
    k = 2

    while current_freq:
        candidates = set()
        for i in range(len(current_freq)):
            for j in range(i + 1, len(current_freq)):
                union = current_freq[i] | current_freq[j]
                if len(union) == k:
                    candidates.add(union)

        candidate_counts = defaultdict(int)
        for tx in transactions:
            tx_set = set(tx)
            for candidate in candidates:
                if candidate.issubset(tx_set):
                    candidate_counts[candidate] += 1

        current_freq = [item for item in candidate_counts if candidate_counts[item] / total_tx >= min_support]
        all_frequent.update({item: candidate_counts[item] for item in current_freq})
        k += 1

    return all_frequent

# ✅ 4. Generate rules
def generate_rules(frequent_itemsets, transactions, min_confidence):
    rules = []
    for itemset in frequent_itemsets:
        if len(itemset) < 2:
            continue
        support_itemset = frequent_itemsets[itemset] / total_tx
        for i in range(1, len(itemset)):
            for antecedent in combinations(itemset, i):
                antecedent = frozenset(antecedent)
                consequent = itemset - antecedent
                support_ante = get_support(antecedent, transactions)
                support_cons = get_support(consequent, transactions)
                if support_ante > 0 and support_cons > 0:
                    confidence = support_itemset / support_ante
                    lift = confidence / support_cons
                    if confidence >= min_confidence:
                        rules.append({
                            'antecedents': set(antecedent),
                            'consequents': set(consequent),
                            'support': round(support_itemset, 2),
                            'confidence': round(confidence, 2),
                            'lift': round(lift, 2)
                        })
    return rules

# ✅ 5. Run everything
min_support = 0.15
min_confidence = 0.2

frequent_itemsets = apriori(transactions, min_support)
rules = generate_rules(frequent_itemsets, transactions, min_confidence)

# ✅ 6. Display
frequent_itemsets_df = pd.DataFrame([{
    'itemsets': set(item),
    'support': round(count / total_tx, 2)
} for item, count in frequent_itemsets.items()])

rules_df = pd.DataFrame(rules)

print("\n📌 Frequent Itemsets:")
print(frequent_itemsets_df)

if not rules_df.empty:
    print("\n📈 Association Rules:")
    print(rules_df[['antecedents', 'consequents', 'support', 'confidence', 'lift']])
else:
    print("\nNo association rules found with confidence ≥", min_confidence)


Q2)

In [1]:
import pandas as pd
from itertools import combinations
from collections import defaultdict

try:
    with open("/content/drive/MyDrive/DataMining/sports (1).txt", "r") as f:
        lines = f.readlines()[1:]
except FileNotFoundError:
    print("Error: 'sports.txt' not found. Please check the file path.")
    raise

transactions_list = []
for line in lines:
    parts = line.strip().split(",")
    items = [item.strip() for item in parts[1:] if item.strip()]
    if items:
        transactions_list.append(items)

def get_support(item, transactions):
    count = sum(1 for transaction in transactions if set(item).issubset(set(transaction)))
    return count / len(transactions) if transactions else 0
print("Sample transactions:", transactions_list[:5])
def fpgrowth(transactions, min_support):

    item_counts = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            item_counts[item] += 1

    total = len(transactions)
    frequent_items = [(item, count/total) for item, count in item_counts.items() if count/total >= min_support]
    frequent_items.sort(key=lambda x: (-x[1], x[0]))
    frequent_itemset = {item for item, _ in frequent_items}

    ordered_transactions = []
    for transaction in transactions:
        ordered = [item for item in transaction if item in frequent_itemset]
        ordered.sort(key=lambda x: (-item_counts[x], x))
        if ordered:
            ordered_transactions.append(ordered)

    patterns = []
    for item, support in frequent_items:

        conditional_transactions = [
            [i for i in t if item_counts[i] >= item_counts[item] and i != item]
            for t in ordered_transactions if item in t
        ]

        for k in range(1, max(len(t) for t in conditional_transactions) + 1):
            candidate_counts = defaultdict(int)
            for t in conditional_transactions:
                for combo in combinations(t, k):
                    candidate_counts[combo] += 1

            for combo, count in candidate_counts.items():

                pattern = sorted(list(combo) + [item])
                support = count / total
                if support >= min_support:
                    patterns.append((pattern, support))


    patterns.extend([([item], support) for item, support in frequent_items])


    unique_patterns = {}
    for pattern, support in patterns:
        key = tuple(sorted(pattern))
        if key not in unique_patterns or unique_patterns[key] < support:
            unique_patterns[key] = support

    return [(list(k), v) for k, v in unique_patterns.items()]

def calc_confidence_lift(frequent_itemsets, transactions):
    print("\n📈 Association Rules:")
    rules_found = False
    for itemset, support in frequent_itemsets:
        if len(itemset) >= 2:
            for i in range(1, len(itemset)):
                for antecedent in combinations(itemset, i):
                    antecedent = list(antecedent)
                    consequent = sorted(list(set(itemset) - set(antecedent)))
                    antecedent_support = get_support(antecedent, transactions)
                    consequent_support = get_support(consequent, transactions)
                    if antecedent_support > 0 and consequent_support > 0:
                        confidence = support / antecedent_support
                        lift = confidence / consequent_support
                        print(f"{antecedent} => {consequent} | support={support:.2f}, confidence={confidence:.2f}, lift={lift:.2f}")
                        rules_found = True
    if not rules_found:
        print("No association rules found. Try lowering min_support or check dataset.")



min_support = 0.1
print("\n=== FP-Growth Results ===")
frequent_patterns = fpgrowth(transactions_list, min_support)
print("📌 Frequent Itemsets from FP-Growth:")
for itemset, support in frequent_patterns:
    print(f"{itemset} - support: {support:.2f}")
calc_confidence_lift(frequent_patterns, transactions_list)

Sample transactions: [['football', 'cricket ball', 'gloves'], ['cricket bat', 'cricket ball', 'juice'], ['football', 'water bottle', 'juice'], ['cricket bat', 'cricket ball', 'gloves'], ['football', 'ice cream']]

=== FP-Growth Results ===
📌 Frequent Itemsets from FP-Growth:
['football', 'juice'] - support: 0.14
['cricket bat', 'juice'] - support: 0.14
['cricket bat', 'football'] - support: 0.12
['cricket ball', 'football'] - support: 0.10
['cricket ball', 'gloves'] - support: 0.14
['cricket ball', 'juice'] - support: 0.12
['cricket ball', 'cricket bat'] - support: 0.14
['football', 'gloves'] - support: 0.14
['cricket bat', 'gloves'] - support: 0.12
['gloves', 'juice'] - support: 0.10
['football', 'water bottle'] - support: 0.10
['football', 'ice cream'] - support: 0.10
['ice cream', 'juice'] - support: 0.10
['football'] - support: 0.44
['juice'] - support: 0.42
['cricket bat'] - support: 0.40
['cricket ball'] - support: 0.36
['gloves'] - support: 0.36
['water bottle'] - support: 0.28


# **Apriori Algorithm**


Uses a bottom-up approach.

Generates candidate itemsets level-by-level (e.g., single items, then pairs, then triples, etc.).

Prunes itemsets that do not meet minimum support.

# **FP-Growth Algorithm**

Uses a divide-and-conquer strategy.

Avoids candidate generation completely.

Builds a compact FP-Tree to store frequent itemsets.

# **Time Complexity**

Apriori:

Worst-case: O(2^n), where n is the number of items.

Overall: O(k⋅n⋅m), where:

k: average size of itemsets, n: number of transactions, m: number of items per transaction.

FP-Growth:

Tree construction: O(n⋅m⋅logm).

Mining time depends on the number and structure of frequent patterns.

Usually faster than Apriori in practice.

# **Space Complexity**

Apriori:

High memory usage.

Stores all candidate itemsets in memory.

FP-Growth:

More memory-efficient.

Uses FP-Tree to compress and store data compactly.



# **Efficiency**

Apriori:

Slower on large or dense datasets.

Requires multiple database scans.

FP-Growth:

Much faster for large datasets.

Only two database scans needed (to build and mine the FP-Tree).